# Demonstration for the upstream regulation network reconstruction

This notebook demonstrates how to use BRAvo to reconstruct a regulation network with specific options and use it as a model, here to infer new knownledge from partial knowledge on nodes evolution.

In [1]:
# Imports
import time
import csv
from IPython.display import display, Markdown, Latex
import networkx as nx
import bravo.regulation as regulation
import bravo.signaling as signaling
import bravo.config as config
import bravo.util as util
import pyBravo as bravo_main

## Network reconstruction

### Global options

We define below the global options regarding the execution:

- Max depth of 10 levels
- Fast reconstruction (look only at standard names)
- Decompose complexes
- Extend search by considering synonyms and removing suffixes
- Include unsinged interactions
- Use Pathway Commons' SPARQL endpoint
- Consider all data sources except MiRTarBase and MSigDB

In [2]:
# Global options

config.MAX_DEPTH = 10
config.HAS_MAX_DEPTH = True
config.FAST = True
config.DECOMPOSE_COMPLEXES = True
config.EXTEND_WITH_SYNONYMS = True
config.EXTEND_WITH_SUFFIXES = True
config.UNKNOWN = True
config.SPARQL_ENDPOINT = "http://rdf.pathwaycommons.org/sparql/"
config.VERBOSE = False

""" all possible data sources """
ds = ['bind', 'biogrid', 'corum',
      'ctd', 'dip', 'drugbank', 'hprd', 'humancyc', 'inoh',
      'intact', 'kegg', 'mirtarbase', 'netpath', 'panther',
      'pid', 'psp', 'reactome', 'reconx', 'smpdb', 'wp',
      'intact_complex', 'msigdb', 'reach', 'wikipathways']

config.DATA_SOURCES = set(ds) - set(['mirtarbase', 'msigdb'])

pybravo_stdout = [''] * 3

### Gene names input list

Here we read the list of input gene names from a file called `input-910.csv`. This file should contain one gene name per line.

In [3]:
# Read input genes list from a file
def read_input_genes(filename):
    res = []
    with open(filename, newline='') as csvfile:
        reader = csv.reader(csvfile, delimiter=' ', quotechar='|')
        for row in reader:
            res.append(''.join(row))
    return res

# Genes input list
gene_list = read_input_genes('reg-demo/data/input-910.csv')
print('{} genes in input list'.format(len(gene_list)))

910 genes in input list


### Initial reconstruction

We now call the reconstruction on the previous list of gene names.
This iterative step is performed by querying the Pathway Commons endpoint and using the results to complete the graph and prepare the next query.
It may take some time, typically between 20 and 30 minutes.

In [4]:
%%capture --no-display pybravo_stdout_0

start_time = time.time()

# Call reconstruction
reconstructed_network = regulation.upstream_regulation(gene_list)

elapsed_time = round((time.time() - start_time), 2)

In [5]:
print("--- Upstream regulation network reconstructed in {} seconds ---".format(elapsed_time))

--- Upstream regulation network reconstructed in 1783.73 seconds ---


In [6]:
%%capture --no-display pybravo_stdout_1

# Build digraph representation in memory
G = bravo_main.build_nx_digraph(reconstructed_network)

bravo_main.write_to_SIF(G, 'reg-demo/out.sif')
bravo_main.write_provenance(G, 'reg-demo/out-provenance.csv')

### Synonym-based unification

We now perform a synonym-unification, which consists in fusing nodes together if they are synonyms.
This process is useful because of the high heterogeneity of the data sources that compose Pathway Commons.
It typically takes a few seconds.

In [7]:
%%capture --no-display pybravo_stdout_2

start_time = time.time()
# Perform unification
G_prime = util.fast_reg_network_unification(G, util.index_syn)
elapsed_time = round((time.time() - start_time), 2)
print("--- Network simplification in %s seconds ---" % elapsed_time)

# Write to files
bravo_main.write_to_SIF(G_prime, 'reg-demo/out-unified.sif')
bravo_main.write_provenance(G_prime, 'reg-demo/out-unified-provenance.csv')

# Count nodes/edges
print('Nodes after simplification = ' + str(len(G_prime.nodes())))
print('Edges after simplification = ' + str(len(G_prime.edges())))

# Compute centrality
print(bravo_main.get_centrality_as_md(G_prime))

### Saving output

Here we simply save all output provided by pyBRAvo during the reconstruction into a texte file.

In [8]:
all_stdout = '\n'.join([s.stdout for s in [pybravo_stdout_0, pybravo_stdout_1, pybravo_stdout_2]])
print(all_stdout)

to be explored after complex decomposition ['LUM', 'CPZ', 'DCN', 'ISLR', 'FNDC1', 'THBS2', 'COL16A1', 'COL1A1', 'COL3A1', 'COL5A1', 'SSC5D', 'MMP28', 'GEM', 'DACT3', 'MRC2', 'GPR124', 'LOXL1', 'ST6GAL2', 'COL1A2', 'PRICKLE1', 'LTBP2', 'COL6A3', 'CDR2L', 'PMP22', 'EMILIN1', 'COL14A1', 'CDH11', 'C1QTNF2', 'CCDC80', 'EFEMP1', 'GLT8D2', 'GGT5', 'ANTXR1', 'SNAI1', 'TSHZ3', 'PPAPDC1A', 'ALDH1A3', 'HEPH', 'FBN1', 'ECM1', 'DPYSL3', 'SRPX', 'PODN', 'GFPT2', 'FOXL1', 'COL10A1', 'GAS1', 'ITGA11', 'GLI2', 'HAND2', 'VCAN', 'GPX8', 'POSTN', 'CRISPLD2', 'PTGIR', 'CALHM2', 'FOXF1', 'MOXD1', 'MGP', 'ASPN', 'COL12A1', 'ZNF469', 'CHST3', 'NHS', 'LAMA2', 'AEBP1', 'PCDH7', 'HTRA3', 'HDGFRP3', 'MXRA5', 'HEYL', 'SVEP1', 'MAP1A', 'SPEG', 'GLI3', 'COL6A2', 'F2R', 'FAP', 'C13orf33', 'SEMA3C', 'ALDH1L2', 'LXN', 'PHLDB1', 'PRELP', 'ADAMTS2', 'PMEPA1', 'EPHA3', 'FLNA', 'RSPO3', 'TGFB3', 'TMEM132E', 'SGCA', 'C1QTNF7', 'COL8A2', 'ADAMTS12', 'MFAP4', 'LAMA1', 'PLAT', 'CTBP2', 'COL4A2', 'NBLA00301', 'BGN', 'LOC1001307

In [9]:
with open('std.out', 'w') as outfile:
    print(all_stdout, file = outfile)

### Statistics extraction

Here we call several Bash and Python scripts on the results to extracts statistics on the provenance and coverage, and we sum up these into a CSV table that can be loaded in Cytoscape and applied to the edges of the network.

In [11]:
%%bash
cd reg-demo

# Extract genes of the unified and non-unified results
sh scripts/get-genes-from-sif.sh out-unified.sif > out-unified.sif.genes
# Provenance of all edges in unified result
bash scripts/pybravo-provenances.sh out-unified-provenance.csv > provenance-unified.txt
# Provenance of signed edges only in unified result
bash scripts/pybravo-provenances.sh <(grep -v 'UNKNOWN' out-unified-provenance.csv) > provenance-unified-signed.txt

# Coverage
python3 scripts/compute_coverage.py out-unified.sif data/input-910.csv ../Homo_sapiens.gene_info > coverage.txt

# Provenance CSV for Cytoscape
mkdir cytoscape
echo "edge name,provenance" > cytoscape/edges-provenance.csv
sed 's/\t\(.*\)\t/ (\1) /;' out-unified-provenance.csv >> cytoscape/edges-provenance.csv

## Using Iggy to make predictions

In this part we use the previously extracted network as a model in order to make predictions on it, using the tool [Iggy](https://github.com/bioasp/iggy).

Using some preliminary observations on some nodes, consisting in signs “$+$” (observed up-regulation) and “$-$” (observed down-regulation), Iggy tries to infer new information on the other nodes which can thus also be predicted as “$+$” or “$-$”. This call relies on Bash and Ptyhon scripts.

For this step, you need Iggy 1.4.1 installed, for instance using pip:

> ```pip install iggy==1.4.1```

Once Iggy is installed, please provide the path to it (default with pip is `~/.local/bin/iggy.py`):

In [12]:
iggy_command = "${HOME}/.local/bin/iggy.py"

In [13]:
# Interpret Iggy command line
iggy_command = !echo "$iggy_command"
iggy_command = iggy_command[0]
print(iggy_command)

/home/max/.local/bin/iggy.py


The scripts below call Iggy, with some data preparation beforehand and a comparison of the result with the original ICGC experimental data afterwards.

The printed output sums up the results of the prediction, and the consistency of these results with the initial ICGC expression data (matching or not matching predictions).

The full result can be found in file `reg-demo/cytoscape/nodes-iggy.csv` in a format that can be fed to Cytoscape in order to label nodes with their observation/prediction sign.

In [14]:
%%bash -s "$iggy_command"
# Make folders for execution results
cd reg-demo
mkdir iggy
cd iggy
mkdir result

# Create graph (SIF) file (removes unsigned edges)
bash ../scripts/format-iggy.sh ../out-unified.sif | grep -v 'UNKNOWN' > out-unified-iggy.sif

# Create observations (OBS) file
bash ../scripts/construct-inputs.sh out-unified-iggy.sif ../data/updownreg-v4.3-noinputs.obs > updownreg.obs

# Extract true observations (actually in network)
bash ../scripts/get-genes-from-sif.sh out-unified-iggy.sif > out-unified-iggy.sif.genes 
grep -w -f out-unified-iggy.sif.genes updownreg.obs > updownreg_true.obs

# Run Iggy workflow
bash ../scripts/workflow-iggy.sh --iggy-command $1 updownreg_true.obs out-unified-iggy.sif ../data/icgc.syn.csv 0 0 result/iggy.out > result/out-unified.tsv
cp result/out-unified.tsv ../cytoscape/nodes-iggy.csv

# Output stats
bash ../scripts/stats-result.sh --en --skip0 result/out-unified.tsv out-unified-iggy.sif > result/stats.txt
cat result/stats.txt

Total number of nodes: 1619
Total number of edges: 4196
Number of observations: 643
Number of predictions: 79
  positive (+): 42
  negative (−): 34
  no-change (0): 0
  may-up (NOT−): 0
  may-down (NOT+): 0
  change (CHANGE/NOT0): 3

Predictions found in ICGC data: 61
    +: 31
    -: 29
    0: 0
  coherent: 29
    +: 20
    -: 9
  weakly coherent: 1
    NOT-: 0
    NOT+: 0
    CHANGE: 1
  not coherent: 31
    predicted +: 11
      (should be -)
     predicted -: 20
      (should be +)
     predicted NOT-/NOT+/CHANGE: 0


In [18]:
%%bash
cd reg-demo
cd iggy
bash ../scripts/stats-2.sh "../out-unified.sif" "../../std.out"

Nodes: 1619 ../out-unified.sif.genes
Edges: 4196 ../out-unified.sif
 - signed: 3625
 - unsigned: 0
 - PART_OF: 571

Coverage: 689 genes are present over 910

Complexes: 304
Complexes involving small molecules: 113
